## Imports

In [46]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Data read-in

In [47]:
df = pd.read_csv('Class A Office DTLA - Sheet1.csv')

## Map prep

In [48]:
# Create full address for geocoding
df['full_address'] = df['Address'] + ' ' + df['DTLA area'] + ' Los Angeles' 

In [49]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [50]:
# Remove NaNs
df = df.dropna(subset='Property name')

In [51]:
df.at[8,'Address']=('505-555 South Flower Street')

In [52]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['full_address'].apply(geocode)

In [53]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [54]:
df.columns

Index(['Property name', 'Address', 'Owner', 'Area', 'Stories', 'Built',
       'Previous valuation', '2022 valuation', 'DTLA area', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'full_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [55]:
def popup_html(row):
    building_name = row['Property name']
    address = row['Address']
    owner = row['Owner']
    area = row['Area']

    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Owner: </strong>{}'''.format(owner) + '''<br>
    <strong>Square Footage: </strong>{}'''.format(area) + '''<br>
    </html>
    '''
    return html

In [61]:
df.at[19,'lat']=(34.0523403990926)
df.at[19,'lon']=(-118.25783008004237)

df.at[8,'lat']=(34.050798764554706)
df.at[8,'lon']=(-118.2572705588038)

df.at[3,'lat']=(34.051372471825054)
df.at[3,'lon']=(-118.25162184866753)

df.at[20,'lat']=(34.04889915280315)
df.at[20,'lon']=(-118.25492148578951)

In [63]:
df

,Property name,Address,Owner,Area,Stories,Built,Previous valuation,2022 valuation,DTLA area,Unnamed: 9,Unnamed: 10,Unnamed: 11,full_address,geocoded,lat,lon
0,Wells Fargo Center,333 South Grand Avenue,Brookfield,"2,571,000",54.0,1982,$556.4 million in 2014,$655.6 million,Bunker Hill,NaN,"$655,600,000",NaN,333 South Grand Avenue Bunker Hill Los Angeles,"(34.0528871, -118.2527252)",34.052887,-118.252725
1,U.S. Bank Tower,633 West 5th Street,Silverstein Properties,"1,432,539",72.0,1989/2023,"$337 million in 2013, sold for $367.5M",$390.9 million,Bunker Hill,NaN,"$390,900,000",NaN,633 West 5th Street Bunker Hill Los Angeles,"(34.0510747, -118.2543536)",34.051075,-118.254354
2,Bank of America Plaza,333 South Hope Street,Brookfield,"1,432,285",55.0,1974,$595 million in 2014,$701 million,Bunker Hill,NaN,"$701,000,000",NaN,333 South Hope Street Bunker Hill Los Angeles,"(34.0534675, -118.2530973)",34.053467,-118.253097
3,Two California Plaza,350 South Grand Avenue,CIM Group,"1,371,385",52.0,1992,"$416 million in 2014, sold for $300.4 million",$490.2 million,Bunker Hill,NaN,"$490,200,000",NaN,350 South Grand Avenue Bunker Hill Los Angeles,"(34.05135, -118.2502183)",34.051350,-118.250218
4,Gas Company Tower,555 West 5th Street,Brookfield (in receivership),"1,303,703",50.0,1991,$487.1 million in 2013,$573 million,Bunker Hill,NaN,"$573,000,000",NaN,555 West 5th Street Bunker Hill Los Angeles,"(34.049929, -118.2529498)",34.049929,-118.252950
5,One California Plaza,300 South Grand Avenue,Rising Realty,"1,039,449",42.0,1985,"$449 million in 2017, sold for $459 million",$500.9 million,Bunker Hill,NaN,"$500,900,000",NaN,300 South Grand Avenue Bunker Hill Los Angeles,"(34.0523179, -118.2514745)",34.052318,-118.251475
6,FourFortyFour South Flower,444 South Flower Street,Oaktree Capital Management,"891,056",48.0,1981,"Sold for $336 million in 2016, assessed at $28...",$318 million,Bunker Hill,NaN,"$318,000,000",NaN,444 South Flower Street Bunker Hill Los Angeles,"(34.0520827, -118.2554134)",34.052083,-118.255413
7,400 South Hope,400 South Hope Street,PNC Financial Services,"701,535",26.0,1982,"Sold for $313.6 million, assessed at $313.5 mi...",$356.7 million,Bunker Hill,NaN,"$356,700,000",NaN,400 South Hope Street Bunker Hill Los Angeles,"(34.0516159, -118.2540899)",34.051616,-118.254090
8,City National Plaza,505-555 South Flower Street,CommonWealth Partners,"2,774,000",52.0,1971/2005,"Sold for $508 million in 2013, assessed at $74...",$874 million,Financial District,NaN,"$874,000,000",NaN,515 South Flower Stret Financial District Los ...,"(34.0364404, -118.2692307)",34.050799,-118.257271
9,Figueroa at Wilshire,601 South Figueroa Street,Brookfield (up for sale),"1,118,941",52.0,1990,$383.5 million in 2014 base value,$451.8 million,Financial District,NaN,"$451,800,000",NaN,601 South Figueroa Street Financial District L...,"(34.0504523, -118.2592131)",34.050452,-118.259213


In [62]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'LA Class A Office Space')

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

for index, row in df.iterrows():
    marker = folium.Marker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html(row), max_width=400))
    marker.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
            
# Display map
m

In [58]:
m.save('index.html')

## Map URL Snagger

In [59]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/ClassA_office_DTLA
